In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from alibi.prototypes import ProtoSelect
from alibi.prototypes.protoselect import cv_protoselect_euclidean, visualize_prototypes
from alibi.utils.kernel import EuclideanDistance
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/robert/miniconda3/envs/alibi/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-33fbd6cbc5fc>", line 4, in <module>
    from alibi.prototypes import ProtoSelect
  File "/home/robert/Desktop/alibi/alibi/__init__.py", line 1, in <module>
    from . import confidence, datasets, explainers, utils
  File "/home/robert/Desktop/alibi/alibi/explainers/__init__.py", line 31, in <module>
    from .shap_wrappers import KernelShap, TreeShap
  File "/home/robert/Desktop/alibi/alibi/explainers/shap_wrappers.py", line 9, in <module>
    import shap
  File "/home/robert/miniconda3/envs/alibi/lib/python3.8/site-packages/shap/__init__.py", line 33, in <module>
    import matplotlib
  File "/home/robert/miniconda3/envs/alibi/lib/python3.8/site-packages/matplotlib/__init__.py", line 109, in <module>
    from . import _api, _version, cbook, docst

TypeError: object of type 'NoneType' has no len()

## Load MNIST dataset

In [ ]:
(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()
X_train, X_test = X_train / 255., X_test / 255.

# get validation
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=1000, random_state=0)

# select random samples from train an test
size = 1000
train_idx = np.random.choice(len(X_train), size=5*size, replace=False)
test_idx = np.random.choice(len(X_test), size=size, replace=False)
X_train, Y_train = X_train[train_idx], Y_train[train_idx]
X_test, Y_test = X_test[test_idx], Y_test[test_idx]

# define preprocessing function
preprocess_fn = lambda x: x.reshape(len(x), -1)

In [ ]:
num_prototypes = 100
grid_size = 20
quantiles = (0.0, 0.5)

# get best eps by cv
eps = cv_protoselect_euclidean(refset=(X_train, Y_train),
                               protoset=(X_test,),
                               valset=(X_val, Y_val),
                               num_splits=5,
                               num_prototypes=num_prototypes,
                               quantiles=quantiles,
                               grid_size=grid_size,
                               preprocess_fn=preprocess_fn)

In [ ]:
eps

In [ ]:
explainer = ProtoSelect(kernel_distance=EuclideanDistance(), eps=eps)
explainer = explainer.fit(X=X_train, X_labels=Y_train, Y=X_test)

In [ ]:
explanation = explainer.explain(num_prototypes=num_prototypes)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn = knn.fit(X=preprocess_fn(explanation.data['prototypes']), y=explanation.data['prototypes_labels'])
knn.score(preprocess_fn(X_test), Y_test)

In [ ]:
len(explanation.data['prototypes'])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)

rand_idx = np.random.choice(len(X_train), size=num_prototypes, replace=False)
protos, labels = X_train[rand_idx], Y_train[rand_idx]
knn = knn.fit(X=preprocess_fn(protos), y=labels)
knn.score(preprocess_fn(X_test), Y_test)

In [ ]:
import umap
reducer = umap.UMAP(random_state=13)
reducer = reducer.fit(preprocess_fn(X_train))

In [ ]:
visualize_prototypes(explanation=explanation,
                     refset=(X_train, Y_train),
                     reducer=reducer.transform,
                     preprocess_fn=preprocess_fn)

## CIFAR10

In [ ]:
!wget https://storage.googleapis.com/seldon-models/alibi/model_cifar10/checkpoint -P model_cifar10
!wget https://storage.googleapis.com/seldon-models/alibi/model_cifar10/cifar10.ckpt.data-00000-of-00001 -P model_cifar10
!wget https://storage.googleapis.com/seldon-models/alibi/model_cifar10/cifar10.ckpt.index -P model_cifar10

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

model = keras.Sequential([
        layers.InputLayer(input_shape=(32, 32, 3)),
        layers.Conv2D(32, (3, 3), kernel_initializer='he_uniform', padding='same'),
        layers.ReLU(),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), kernel_initializer='he_uniform', padding='same'),
        layers.ReLU(),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.2),
        layers.Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same'),
        layers.ReLU(),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same'),
        layers.ReLU(),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        layers.Conv2D(128, (3, 3), kernel_initializer='he_uniform', padding='same'),
        layers.ReLU(),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), kernel_initializer='he_uniform', padding='same'),
        layers.ReLU(),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.4),
        layers.Flatten(),
        layers.Dense(128, kernel_initializer='he_uniform', name='feature_layer'),
        layers.ReLU(),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(10)
    ])

In [ ]:
model.load_weights('model_cifar10/cifar10.ckpt')